In [1]:
import os
import sys
import pandas as pd

INPUT_PATH = "全国大学数据.csv"
OUTPUT_PATH = "全国大学数据_筛选.csv"

KEEP_COLUMNS = ["省份", "大学", "本或专科", "985", "211", "双一流", "城市"]

def map_12_to_10(value):
    if pd.isna(value):
        return 0
    v = str(value).strip()
    if v == "1":
        return 1
    if v == "2":
        return 0
    # 其他情况，尽量按数字处理；否则归零
    try:
        num = int(float(v))
        return 1 if num == 1 else 0
    except Exception:
        return 0

def map_double_first_class(value):
    if pd.isna(value):
        return 0
    v = str(value).strip()
    # 只要包含“双一流”字样即认为是1，否则0（包括空字符串）
    return 1 if ("双一流" in v and len(v) > 0) else 0

def main():
    # 读取 CSV（自动尝试常见编码）
    encodings = ["utf-8", "utf-8-sig", "gbk"]
    last_err = None
    df = None
    for enc in encodings:
        try:
            df = pd.read_csv(INPUT_PATH, encoding=enc)
            break
        except Exception as e:
            last_err = e
    if df is None:
        print(f"读取失败：{last_err}")
        sys.exit(1)

    # 只保留指定列（若缺列则报错提示）
    missing = [c for c in KEEP_COLUMNS if c not in df.columns]
    if missing:
        print(f"缺少必要列：{missing}，请检查文件表头是否为：{KEEP_COLUMNS}")
        sys.exit(1)
    df = df[KEEP_COLUMNS].copy()

    # 标准化 985 / 211
    df["985"] = df["985"].apply(map_12_to_10).astype(int)
    df["211"] = df["211"].apply(map_12_to_10).astype(int)

    # 标准化 双一流
    df["双一流"] = df["双一流"].apply(map_double_first_class).astype(int)

    # 仅保留 “普通本科”
    df = df[df["本或专科"].astype(str).str.strip() == "普通本科"]

    # 删除 985/211/双一流 同时为 0 的行
    df = df[~((df["985"] == 0) & (df["211"] == 0) & (df["双一流"] == 0))]

    # 导出
    out_dir = os.path.dirname(OUTPUT_PATH)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
    print(f"处理完成，共 {len(df)} 行，已导出：{OUTPUT_PATH}")

if __name__ == "__main__":
    main()

处理完成，共 150 行，已导出：全国大学数据_筛选.csv
